In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("/kaggle/input/playground-series-s4e10/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e10/test.csv")

In [4]:
test.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [5]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

In [7]:
duplicate = df[df["id"].duplicated(keep=False)]
print(duplicate["id"].unique())

#  there is nobody who takes loan multiple times

[]


In [8]:
df["person_home_ownership"].value_counts()

person_home_ownership
RENT        30594
MORTGAGE    24824
OWN          3138
OTHER          89
Name: count, dtype: int64

In [9]:
df["loan_grade"].value_counts()
#df["loan_intent"].value_counts()
#df["cb_person_default_on_file"].value_counts()

loan_grade
A    20984
B    20400
C    11036
D     5034
E     1009
F      149
G       33
Name: count, dtype: int64

In [10]:
x_train = df.drop(columns=['loan_status','id'])
x_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3


In [11]:
y_train = df['loan_status']
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

# steps
 apply ohe in person_home_ownership, loan_intent, loan_grade , lebel encoding on cb_person_default_on_file  , standard_scaler on loan_amnt and person_income , use pipelines

In [12]:
# imporing necessary algorithms

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [13]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first')
label = OrdinalEncoder()
scale = StandardScaler()


In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', ohe, [2, 4, 5]),   # One-hot encode 
        ('label', label, [9]),       #  label encode
        ('scaler', scale, [1, 6])  # Standard scaling for numerical columns
    ],
    remainder='passthrough' )

In [15]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=500,random_state=42))
])

In [16]:
pipe2 =  Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(max_features=200,max_depth=4,bootstrap=True,oob_score=True,random_state=42))
])

In [17]:
pipe3 =  Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=200,max_depth=4,learning_rate=0.1,random_state=42,subsample=0.9,))
])

In [18]:
pipe4 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier1', xgb.XGBClassifier(n_estimators=200,max_depth=5,learning_rate=0.1,random_state=42,objective='binary:logistic'))
])

In [19]:
pipe5 = Pipeline(steps=[
    ('preprocessor', preprocessor)])

In [20]:
pipe.fit(x_train,y_train)
pipe2.fit(x_train,y_train)
pipe3.fit(x_train,y_train)
pipe4.fit(x_train,y_train)


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [2, 4, 5]),
                                                 ('label', OrdinalEncoder(),
                                                  [9]),
                                                 ('scaler', StandardScaler(),
                                                  [1, 6])])),
                ('classifier1',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsamp...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [21]:
x_out = pipe.predict(x_train)
x_out2 = pipe2.predict(x_train)
x_out3= pipe3.predict(x_train)
x_out4= pipe4.predict(x_train)

In [22]:
accuracy_score(y_train,x_out)*100

91.17571830505584

In [23]:
accuracy_score(y_train,x_out2)*100

93.58342569699036

In [24]:
accuracy_score(y_train,x_out3)*100

95.55290306078949

In [25]:
accuracy_score(y_train,x_out4)*100

95.67738085088243

In [26]:
#  hyper parameter tuning using xgboost
parameters = {
    'classifier1__max_depth': [3, 6, 9,10],
    'classifier1__learning_rate': [0.01, 0.05, 0.1, 0.2,0.5],
    'classifier1__n_estimators': [50, 100, 200,300],
    'classifier1__subsample': [0.8, 0.9, 1.0,0.5],       
    'classifier1__colsample_bytree': [0.6, 0.8, 1.0,0.5]}
hyper_xgb = GridSearchCV(pipe4, param_grid=parameters ,scoring="accuracy",n_jobs=-1,cv=4,verbose=3)

In [27]:
from sklearn.model_selection import RandomizedSearchCV

In [28]:
randm_cv = RandomizedSearchCV(pipe4, param_distributions=parameters ,scoring="accuracy",n_jobs=-1,n_iter=50,cv=4,verbose=3,random_state=42)

In [29]:
new_xgb = randm_cv.fit(x_train,y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
[CV 1/4] END classifier1__colsample_bytree=0.5, classifier1__learning_rate=0.5, classifier1__max_depth=6, classifier1__n_estimators=200, classifier1__subsample=0.8;, score=0.944 total time=   3.0s
[CV 1/4] END classifier1__colsample_bytree=1.0, classifier1__learning_rate=0.01, classifier1__max_depth=10, classifier1__n_estimators=100, classifier1__subsample=0.5;, score=0.946 total time=   2.6s
[CV 1/4] END classifier1__colsample_bytree=1.0, classifier1__learning_rate=0.5, classifier1__max_depth=3, classifier1__n_estimators=50, classifier1__subsample=0.5;, score=0.947 total time=   0.8s
[CV 1/4] END classifier1__colsample_bytree=1.0, classifier1__learning_rate=0.1, classifier1__max_depth=3, classifier1__n_estimators=50, classifier1__subsample=1.0;, score=0.939 total time=   0.8s
[CV 4/4] END classifier1__colsample_bytree=1.0, classifier1__learning_rate=0.1, classifier1__max_depth=3, classifier1__n_estimators=50, classifier1__s

In [30]:
x_out4= new_xgb.predict(x_train)
accuracy_score(y_train,x_out4)*100

96.87782419643618

In [31]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense

In [32]:
x_train_transform = pipe5.fit_transform(x_train)

In [33]:
model = Sequential([
    Dense(units=20, activation='relu'),
    Dense(units=10, activation='relu'),
    Dense(units=5, activation='relu'),
    Dense(units=1, activation='sigmoid')    
])
model.compile(loss=BinaryCrossentropy(),optimizer='adam',metrics=['accuracy'])
             
model.fit(x_train_transform,y_train,epochs=25)

Epoch 1/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8771 - loss: 0.3258
Epoch 2/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9082 - loss: 0.2497
Epoch 3/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9117 - loss: 0.2453
Epoch 4/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9176 - loss: 0.2312
Epoch 5/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9261 - loss: 0.2177
Epoch 6/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9289 - loss: 0.2073
Epoch 7/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9309 - loss: 0.2076
Epoch 8/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9300 - loss: 0.2096
Epoch 9/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9333 - loss: 0.2046
Epoch 10/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9342 - loss: 0.2048
Epoch 11/25
1833/1833 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9351 - loss: 0.1979
Epoch 12/25
1833/1833 ━━━━━━━━

In [34]:
y_train_transform = model.predict(x_train_transform)
threshold=0.4
new_y_train_transform = [1 if x>threshold else 0 for x in y_train_transform]

1833/1833 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [35]:
accuracy_score(y_train,new_y_train_transform)*100

93.92957626396112

In [36]:
y_out = randm_cv.predict_proba(test.drop(columns=['id']))[:,1]
y_out = pd.DataFrame(y_out,columns=['loan_status'])
y_out['id'] = test['id']
submission = y_out[['id','loan_status']] 
submission.set_index('id',inplace=True)
submission.to_csv('submission_xgb.csv')